In [1]:
# guardar_como: replicar_cnn.py
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import numpy as np
import pandas as pd
import random, os
import shutil
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split

# ----------------------------------------------------------------------
# 1. CONFIGURACIÓN: ¡CAMBIA ESTA RUTA!
# ----------------------------------------------------------------------
# Apunta esta variable a la carpeta que contiene el 'train.csv' y 
# la carpeta 'gaussian_filtered_images'.
#
# Ejemplo de estructura de carpetas esperada:
# DATA_ROOT_DIR/
# |-- train.csv
# |-- gaussian_filtered_images/
#     |-- gaussian_filtered_images/
#         |-- No_DR/
#         |-- Mild/
#         |-- ...
#
DATA_ROOT_DIR = r'../data/2'


2025-11-06 21:16:21.127763: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
# ----------------------------------------------------------------------
# 2. PREPARACIÓN DE DATOS (DataFrame y Mapeo)
# ----------------------------------------------------------------------
print("Paso 1: Cargando y preparando los datos del CSV...")
csv_path = os.path.join(DATA_ROOT_DIR, 'train.csv')
df = pd.read_csv(csv_path)

# Mapeo binario: 0 = No_DR, 1-4 = DR
diagnosis_dict_binary = {
    0: 'No_DR', 1: 'DR', 2: 'DR', 3: 'DR', 4: 'DR'
}
# Mapeo original (para estratificación)
diagnosis_dict_original = {
    0: 'No_DR', 1: 'Mild', 2: 'Moderate', 3: 'Severe', 4: 'Proliferate_DR'
}

df['binary_type'] = df['diagnosis'].map(diagnosis_dict_binary.get)
df['type'] = df['diagnosis'].map(diagnosis_dict_original.get)

Paso 1: Cargando y preparando los datos del CSV...


In [3]:
# ----------------------------------------------------------------------
# 3. DIVISIÓN DE DATOS (Train/Val/Test)
# ----------------------------------------------------------------------
print("Paso 2: Dividiendo los datos en sets de entrenamiento, validación y prueba...")
# Dividir 70% train, 15% val, 15% test
train_intermediate, val = train_test_split(df, test_size=0.15, stratify=df['type'])
train, test = train_test_split(train_intermediate, test_size=(0.15 / (1 - 0.15)), stratify=train_intermediate['type'])

print(f"Total de entrenamiento: {len(train)}")
print(f"Total de validación: {len(val)}")
print(f"Total de prueba: {len(test)}")

Paso 2: Dividiendo los datos en sets de entrenamiento, validación y prueba...
Total de entrenamiento: 2562
Total de validación: 550
Total de prueba: 550


In [4]:
# ----------------------------------------------------------------------
# 4. CREAR DIRECTORIOS Y COPIAR IMÁGENES
# ----------------------------------------------------------------------
print("Paso 3: Creando directorios locales y copiando imágenes (esto puede tardar)...")

# Directorio base local donde se crearán las carpetas 'train', 'val', 'test'
# Puedes cambiar 'data_cnn' por el nombre que quieras
base_dir = 'data_cnn'

train_dir = os.path.join(base_dir, 'train')
val_dir = os.path.join(base_dir, 'val')
test_dir = os.path.join(base_dir, 'test')
src_dir = os.path.join(DATA_ROOT_DIR, 'gaussian_filtered_images/gaussian_filtered_images')

# Limpiar directorios si existen
if os.path.exists(base_dir):
    shutil.rmtree(base_dir)
os.makedirs(train_dir)
os.makedirs(val_dir)
os.makedirs(test_dir)

# Función auxiliar para copiar archivos
def copy_files(dataframe, dest_folder):
    for index, row in dataframe.iterrows():
        # 'type' es la carpeta original (ej. 'Mild')
        original_diagnosis_folder = row['type']
        # 'binary_type' es la nueva carpeta (ej. 'DR')
        binary_diagnosis_folder = row['binary_type']
        
        id_code = row['id_code'] + ".png"
        
        # Ruta de origen
        srcfile = os.path.join(src_dir, original_diagnosis_folder, id_code)
        
        # Ruta de destino
        dstfile_dir = os.path.join(dest_folder, binary_diagnosis_folder)
        os.makedirs(dstfile_dir, exist_ok=True)
        
        # Copiar
        if os.path.exists(srcfile):
            shutil.copy(srcfile, dstfile_dir)
        else:
            print(f"Advertencia: Archivo no encontrado {srcfile}")

# Ejecutar la copia
print("Copiando imágenes de entrenamiento...")
copy_files(train, train_dir)
print("Copiando imágenes de validación...")
copy_files(val, val_dir)
print("Copiando imágenes de prueba...")
copy_files(test, test_dir)

print("¡Copia de archivos completada!")

Paso 3: Creando directorios locales y copiando imágenes (esto puede tardar)...
Copiando imágenes de entrenamiento...
Copiando imágenes de validación...
Copiando imágenes de prueba...
¡Copia de archivos completada!


In [5]:
# ----------------------------------------------------------------------
# 5. GENERADORES DE IMÁGENES
# ----------------------------------------------------------------------
print("Paso 4: Configurando los generadores de datos (ImageDataGenerator)...")
train_path = train_dir
val_path = val_dir
test_path = test_dir

# Rescala las imágenes (normalización)
train_batches = ImageDataGenerator(rescale=1./255).flow_from_directory(
    train_path, 
    target_size=(224, 224), 
    shuffle=True
)
val_batches = ImageDataGenerator(rescale=1./255).flow_from_directory(
    val_path, 
    target_size=(224, 224), 
    shuffle=True
)
test_batches = ImageDataGenerator(rescale=1./255).flow_from_directory(
    test_path, 
    target_size=(224, 224), 
    shuffle=False  # Importante para la evaluación
)

Paso 4: Configurando los generadores de datos (ImageDataGenerator)...
Found 2562 images belonging to 2 classes.
Found 550 images belonging to 2 classes.
Found 550 images belonging to 2 classes.


In [6]:

# ----------------------------------------------------------------------
# 6. CONSTRUCCIÓN DEL MODELO CNN
# ----------------------------------------------------------------------
print("Paso 5: Construyendo el modelo CNN...")
model = tf.keras.Sequential([
    layers.Conv2D(8, (3,3), padding="valid", input_shape=(224,224,3), activation='relu'),
    layers.MaxPooling2D(pool_size=(2,2)),
    layers.BatchNormalization(),
    
    layers.Conv2D(16, (3,3), padding="valid", activation='relu'),
    layers.MaxPooling2D(pool_size=(2,2)),
    layers.BatchNormalization(),
    
    layers.Conv2D(32, (4,4), padding="valid", activation='relu'),
    layers.MaxPooling2D(pool_size=(2,2)),
    layers.BatchNormalization(),
 
    layers.Flatten(),
    layers.Dense(32, activation='relu'),
    layers.Dropout(0.15),
    layers.Dense(2, activation='softmax') # 2 Salidas: 'No_DR' y 'DR'
])

model.summary()

Paso 5: Construyendo el modelo CNN...


/usr/local/python/3.12.1/lib/python3.12/site-packages/keras/src/layers/convolutional/base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
2025-11-06 21:17:18.499658: E external/local_xla/xla/stream_executor/cuda/cuda_platform.cc:51] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: UNKNOWN ERROR (303)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 222, 222, 8)    │           224 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 111, 111, 8)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization             │ (None, 111, 111, 8)    │            32 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 109, 109, 16)   │         1,168 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 54, 54, 16)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_1           │ (None, 54, 54, 16)     │            64 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 51, 51, 32)     │         8,224 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 25, 25, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_2           │ (None, 25, 25, 32)     │           128 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 20000)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 32)             │       640,032 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 32)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 2)              │            66 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 649,938 (2.48 MB)

 Trainable params: 649,826 (2.48 MB)

 Non-trainable params: 112 (448.00 B)

In [7]:
# ----------------------------------------------------------------------
# 7. COMPILACIÓN Y ENTRENAMIENTO
# ----------------------------------------------------------------------
print("Paso 6: Compilando y entrenando el modelo...")
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-5),
              loss=tf.keras.losses.BinaryCrossentropy(),
              metrics=['acc'])

history = model.fit(train_batches,
                    epochs=30, # El original usa 30, puedes bajarlo a 5-10 para pruebas
                    validation_data=val_batches)

print("¡Entrenamiento completado!")


Paso 6: Compilando y entrenando el modelo...
Epoch 1/30


2025-11-06 21:17:32.933877: W external/local_xla/xla/tsl/framework/cpu_allocator_impl.cc:84] Allocation of 50466816 exceeds 10% of free system memory.


 1/81 ━━━━━━━━━━━━━━━━━━━━ 3:26 3s/step - acc: 0.4688 - loss: 0.9309

2025-11-06 21:17:33.248404: W external/local_xla/xla/tsl/framework/cpu_allocator_impl.cc:84] Allocation of 50466816 exceeds 10% of free system memory.
2025-11-06 21:17:33.355738: W external/local_xla/xla/tsl/framework/cpu_allocator_impl.cc:84] Allocation of 50466816 exceeds 10% of free system memory.


 2/81 ━━━━━━━━━━━━━━━━━━━━ 33s 423ms/step - acc: 0.4219 - loss: 0.9500

2025-11-06 21:17:33.650474: W external/local_xla/xla/tsl/framework/cpu_allocator_impl.cc:84] Allocation of 50466816 exceeds 10% of free system memory.
2025-11-06 21:17:33.778226: W external/local_xla/xla/tsl/framework/cpu_allocator_impl.cc:84] Allocation of 50466816 exceeds 10% of free system memory.


81/81 ━━━━━━━━━━━━━━━━━━━━ 35s 399ms/step - acc: 0.8411 - loss: 0.4046 - val_acc: 0.5073 - val_loss: 0.7677
Epoch 2/30
81/81 ━━━━━━━━━━━━━━━━━━━━ 32s 396ms/step - acc: 0.9110 - loss: 0.2564 - val_acc: 0.5073 - val_loss: 1.0385
Epoch 3/30
81/81 ━━━━━━━━━━━━━━━━━━━━ 31s 383ms/step - acc: 0.9290 - loss: 0.2291 - val_acc: 0.5073 - val_loss: 1.3041
Epoch 4/30
81/81 ━━━━━━━━━━━━━━━━━━━━ 32s 392ms/step - acc: 0.9317 - loss: 0.2150 - val_acc: 0.5073 - val_loss: 1.5781
Epoch 5/30
81/81 ━━━━━━━━━━━━━━━━━━━━ 31s 388ms/step - acc: 0.9426 - loss: 0.2030 - val_acc: 0.5255 - val_loss: 1.1920
Epoch 6/30
81/81 ━━━━━━━━━━━━━━━━━━━━ 38s 354ms/step - acc: 0.9411 - loss: 0.1936 - val_acc: 0.6218 - val_loss: 0.7610
Epoch 7/30
81/81 ━━━━━━━━━━━━━━━━━━━━ 31s 383ms/step - acc: 0.9415 - loss: 0.1825 - val_acc: 0.8436 - val_loss: 0.3277
Epoch 8/30
81/81 ━━━━━━━━━━━━━━━━━━━━ 31s 385ms/step - acc: 0.9457 - loss: 0.1765 - val_acc: 0.8945 - val_loss: 0.2333
Epoch 9/30
81/81 ━━━━━━━━━━━━━━━━━━━━ 31s 384ms/step - acc:

In [8]:
# ----------------------------------------------------------------------
# 8. GUARDAR MODELO Y EVALUAR
# ----------------------------------------------------------------------
print("Paso 7: Guardando y evaluando el modelo...")
model.save("cnn_model_94.h5")
print("Modelo guardado como 'cnn_model_94.h5'")

print("Evaluando en el set de prueba (test_batches)...")
loss, acc = model.evaluate(test_batches, verbose=1)
print("-" * 30)
print(f"Precisión en datos de prueba: {acc * 100:.2f}%")
print(f"Pérdida en datos de prueba: {loss:.4f}")
print("-" * 30)

Paso 7: Guardando y evaluando el modelo...
Modelo guardado como 'cnn_model_94.h5'
Evaluando en el set de prueba (test_batches)...
18/18 ━━━━━━━━━━━━━━━━━━━━ 2s 127ms/step - acc: 0.9345 - loss: 0.1696
------------------------------
Precisión en datos de prueba: 93.45%
Pérdida en datos de prueba: 0.1696
------------------------------
